# Main part

## Load and clean data

Import of used packages.

In [ ]:
import numpy as np
import pandas as pd
import datetime
from tqdm.notebook import tqdm
from random import sample
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import tree
from sklearn.metrics import roc_auc_score
import random


Importing data. hw1_devsample and hw1_outofsample

In [ ]:
random.seed(10) 

from functions import *
# Load data - set index column, decimal point, separator
data = pd.read_csv('hw1_devsample.csv', sep=',',
                   decimal='.', index_col='SK_ID_CURR')

# print time of data being loaded - use strftime
print(f'Data loaded on:   {datetime.datetime.now().strftime(format="%Y-%m-%d %H:%M:%S")}')
data_cart = data.copy()

data_test = pd.read_csv('hw1_outofsample.csv', sep=',',decimal='.', index_col='SK_ID_CURR')

We remove variables with more than 30% NA.

We also remove the time variables.

In [ ]:
del_col = []
for i in range(0, len(data_cart.isna().sum())):
    if data_cart[data_cart.columns[i]].isna().sum()/80000 > 0.3:
        del_col.append(data_cart.columns[i])
data_cart = data_cart.drop(del_col, axis=1)
data_cart = data_cart.drop(['MONTH','TIME','DAY','BASE'], axis=1)

data_test = data_test.drop(del_col, axis=1)
data_test = data_test.drop(['MONTH','TIME','DAY','BASE'], axis=1)

data_cart_fillna = data_cart.copy()
data_cart_na = data_cart.copy()

data_test_na = data_test.copy()

Because we want to predict on the file on the data from the "hw1_outofsample.csv" file, we want our model to train on the same data. Therefore, we want the shape of the two datasets similar.

In [ ]:
data_cart_na = data_cart_na.dropna(axis = 0)
data_cart_na = pd.get_dummies(data_cart_na[data_cart_na.columns])

data_test = data_test.dropna(axis = 1)
data_test = pd.get_dummies(data_test[data_test.columns])

In [ ]:
print(data_cart_na.info())
print(data_test.info())

we store the variable names of the data in a list.

In [ ]:
cols_pred = list(data_cart_na.columns)
cols_test = list(data_test.columns)

Find common variables between the train data and the test data.

In [ ]:
col_similar = []
for i in cols_pred:
    for j in cols_test:
        if i==j:
            col_similar.append(i)

In [ ]:
col_similar=list(set(cols_pred).intersection(cols_test))
print(len(col_similar))

Keep only common variables in the data_test.

In addition to this, we add the TARGET to the list of variable names.

In [ ]:
data_test = data_test[col_similar]
col_similar_target = col_similar + ['TARGET']

We only keep the variables and target similar to the training data. 

In [ ]:
data_cart_na = data_cart_na[col_similar_target]

The first tree is trained using randomly chosen subset of train dataset (we denote data_fraction the fraction of overall train dataset). Similarly, only predictor_fraction fraction of available predictors is used. The remaining trees are then trained as follows:

In [ ]:
target = data_cart_na['TARGET'].astype(int, copy=False)
data_cart_iter = data_cart_na.drop(['TARGET'], axis = 1)

## Trees

Tree construction criteria :

In [ ]:
criterion = 'gini' #(TODO)
splitter = 'best' #(TODO)
mdepth = 20 #(TODO)
min_samples_split = 5 #(TODO)
min_samples_leaf = 0.0001 #(TODO)
minleaf = 100 #(TODO)
random_state = 1
max_features = 0.8

Creation of our first split data :

In [ ]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(data_cart_iter, target, test_size=0.25)
print(X_train1.shape[0],X_train1.shape[1])

Tree creation and training

In [ ]:
model = tree.DecisionTreeClassifier(
    criterion=criterion,
    splitter=splitter,
    max_depth=mdepth,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    random_state=random_state,
    max_features = max_features
)

# And fit training data.
clf = model.fit(X_train1, Y_train1)

Calculate the yield of this tree. 

In [ ]:
pred_train = model.predict_proba(X_train1)[:,1]
pred_test = model.predict_proba(X_test1)[:,1]
data_cart_na['Pred'] = 0
data_cart_na.loc[list(X_train1.index),'Pred'] = pred_train
data_cart_na.loc[list(X_test1.index),'Pred'] = pred_test

data_cart_na['Pred_label'] = (data_cart_na['Pred'] >= 0.5).astype(int)

data_cart_na['Res'] = abs(data_cart_na['TARGET']-data_cart_na['Pred'])

prediction = model.predict_proba(data_test)[:,1]

In [ ]:
def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq: http://www.statsdirect.com/help/content/image/stat0206_wmf.gif
    # from: http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    array = array.flatten() #all values are treated equally, arrays must be 1d
    if np.amin(array) < 0:
        array -= np.amin(array) #values cannot be negative
    array += 0.0000001 #values cannot be 0
    array = np.sort(array) #values must be sorted
    index = np.arange(1,array.shape[0]+1) #index per array element
    n = array.shape[0]#number of array elements
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array))) #Gini coefficient

In [ ]:
gini_bs = gini(prediction)
gini_bs

Here we create a loop to improve the performance of our tree. Iteration by iteration, the weights of the individuals are weighted by the results of the previous tree. This is to improve the AUC.  

In [ ]:
from sklearn.metrics import roc_auc_score
n = 10
clf = []
for n in range(0,n):
    
    X_train_i, X_test_i, Y_train_i, Y_test_i = train_test_split(data_cart_iter, target, test_size=0.25)

    model = tree.DecisionTreeClassifier(
    criterion=criterion,
    splitter=splitter,
    max_depth=mdepth,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    max_features = max_features,
    random_state=n
    )

    # And fit training data.
    #clf_i = model.fit(X_train_i, Y_train_i, sample_weight = data_cart_na.loc[list(X_train_i.index),'Pred'])
    clf_i = model.fit(X_train_i, Y_train_i, sample_weight = gini_bs)

    
    pred_train = model.predict_proba(X_train_i)[:,1]
    pred_test = model.predict_proba(X_test_i)[:,1]
    
    data_cart_na.loc[list(X_train_i.index),'Pred'] = (pred_train + (n*data_cart_na.loc[list(X_train_i.index),'Pred']))/(n+1)
    data_cart_na.loc[list(X_test_i.index),'Pred'] = (pred_test + (n*data_cart_na.loc[list(X_test_i.index),'Pred']))/(n+1)

    data_cart_na['Pred_label'] = (data_cart_na['Pred'] >= 0.5).astype(int)
    
    data_cart_na['Res'] = abs(data_cart_na['TARGET'] - data_cart_na['Pred'])
    
    print('*************** Evaluation After Iteration '+str(n+1)+' *********')
    
    #score_te = model.score(X_test_i, Y_test_i)
    #print('Accuracy Score: ', score_te)
    
    #pred_labels_test = model.predict_proba(X_test_i) 
    #print('Taux derreur', 1.0 - metrics.accuracy_score(Y_test_i,pred_labels_test))

    prediction = (model.predict_proba(data_test)[:,1] + (n*prediction))/(n+1)

    gini_bs = gini(prediction)
    print('Gini' , gini_bs)
    print('AUC',roc_auc_score(Y_test_i, data_cart_na.loc[list(Y_test_i.index),'Pred'])) 


The AUC score that our algorithm gives us is very good. 

Predicting target values for the test data with our forest.

## Predict

We save our results in a csv file

In [ ]:
data_saving = pd.DataFrame({'SK_ID_CURR' : data_test.index ,'prediction' : prediction})
data_saving.to_csv('DS2_22_HW1_CADIOU&PAIN.csv',index=False,sep=',')
data_saving.info()

# Annex

In the beginning, we created a copy of data_cart namd data_cart_fillna. It was created to propose a different version of handling missing values. 

In the following code, we do exactly the same as before but by giving a median value to the NA's. However, we obtain results that seems to be not much interesting.

In [ ]:
data_cart_fillna = pd.get_dummies(data_cart_fillna[data_cart_fillna.columns])
data_cart_fillna= data_cart_fillna.fillna(data_cart_fillna.median())
data_test_na = pd.get_dummies(data_test_na[data_test_na.columns])
data_test_na = data_test_na.fillna(data_test_na.median())

In [ ]:
cols_pred_na = list(data_cart_fillna.columns)
cols_test_na = list(data_test_na.columns)
col_similar_na = []
for i in cols_pred_na:
    for j in cols_test_na:
        if i==j:
            col_similar_na.append(i)
col_similar_na=list(set(cols_pred_na).intersection(cols_test_na))
print(len(col_similar_na))

In [ ]:
data_test_na = data_test_na[col_similar_na]
col_similar_target_na = col_similar_na + ['TARGET']
data_cart_fillna = data_cart_fillna[col_similar_target_na]

In [ ]:
target_na = data_cart_fillna['TARGET'].astype(int, copy=False)
data_cart_iter_fillna = data_cart_fillna.drop(['TARGET'], axis = 1)

criterion = 'gini' #(TODO)
splitter = 'best' #(TODO)
mdepth = 20 #(TODO)
min_samples_split = 5 #(TODO)
min_samples_leaf = 0.0001 #(TODO)
minleaf = 100 #(TODO)
random_state = 1
max_features = 0.8

X_train1_na, X_test1_na, Y_train1_na, Y_test1_na = train_test_split(data_cart_iter_fillna, target_na, test_size=0.25)
print(X_train1_na.shape[0],X_train1_na.shape[1])

In [ ]:
model_na = tree.DecisionTreeClassifier(
    criterion=criterion,
    splitter=splitter,
    max_depth=20,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    random_state=random_state,
    max_features = max_features
)

# And fit training data.
clf_na = model_na.fit(X_train1_na, Y_train1_na)

pred_train_na = model_na.predict_proba(X_train1_na)[:,1]
pred_test_na = model_na.predict_proba(X_test1_na)[:,1]
data_cart_fillna['Pred'] = 0
data_cart_fillna.loc[list(X_train1_na.index),'Pred'] = pred_train_na
data_cart_fillna.loc[list(X_test1_na.index),'Pred'] = pred_test_na

data_cart_fillna['Pred_label'] = (data_cart_fillna['Pred'] >= 0.5).astype(int)

data_cart_fillna['Res'] = abs(data_cart_fillna['TARGET']-data_cart_fillna['Pred'])

prediction_na = model_na.predict_proba(data_test_na)[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score
n = 10
clf_na = []
for n in range(0,n):
    
    X_train_i, X_test_i, Y_train_i, Y_test_i = train_test_split(data_cart_iter_fillna, target_na, test_size=0.25)

    model = tree.DecisionTreeClassifier(
    criterion=criterion,
    splitter=splitter,
    max_depth=20,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    max_features = max_features,
    random_state=n
    )

    # And fit training data.
    clf_i = model.fit(X_train_i, Y_train_i, sample_weight = data_cart_fillna.loc[list(X_train_i.index),'Pred'])
    
    pred_train = model.predict_proba(X_train_i)[:,1]
    pred_test = model.predict_proba(X_test_i)[:,1]
    
    data_cart_fillna.loc[list(X_train_i.index),'Pred'] = (pred_train + (n*data_cart_fillna.loc[list(X_train_i.index),'Pred']))/(n+1)
    data_cart_fillna.loc[list(X_test_i.index),'Pred'] = (pred_test + (n*data_cart_fillna.loc[list(X_test_i.index),'Pred']))/(n+1)

    data_cart_fillna['Pred_label'] = (data_cart_fillna['Pred'] >= 0.5).astype(int)
    
    data_cart_fillna['Res'] = abs(data_cart_fillna['TARGET'] - data_cart_fillna['Pred'])
    
    print('*************** Evaluation After Iteration '+str(n+1)+' *********')
    
    prediction = (model.predict_proba(data_test_na)[:,1] + (n*prediction))/(n+1)


    print('AUC',roc_auc_score(Y_test_i, data_cart_fillna.loc[list(Y_test_i.index),'Pred_label'])) 


In [ ]:
y_pred_proba = model.predict(data_test_na)
data_saving = pd.DataFrame({'SK_ID_CURR' : data_test_na.index ,'prediction' : y_pred_proba})
data_saving.to_csv('DS2_22_HW1_CADIOU&PAIN.csv',index=False,sep=',')
data_saving.info()